In [6]:
#Определяется конфигурация библиотек Python. В дальнейшем желательно вынести в отдельный конфигурационный файл
import gensim
from gensim import models
from gensim import corpora
from pprint import pprint
import os
from gensim.utils import simple_preprocess
from smart_open import smart_open
import numpy as np


#Выгрузка stopwords корпуса слов 
import nltk
nltk.download('stopwords')  # запускается один раз
# Установка лемманизации
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

#Cоздание списка лемманизации и stopwords
mystem = Mystem() 
stop_words = stopwords.words("russian")

#Функция лемманизации
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in stop_words \
              and token != " " \
              and token.strip() not in punctuation]
    
    text = ' '.join(tokens)
    return text
#   возвращает текст
#Пример    
#print(preprocess_text('Ну что сказать, я вижу кто-то наступил на грабли, Ты разочаровал меня, ты был натравлен.'))
#> 'сказать видеть кто-то наступать грабли разочаровывать натравлять'

#установка путей и конфигураций
path_to_text_file_directory = 'D:\\archive'
archive_file_name = '01_text_utf_8.txt'  #'01_text_utf_8.txt'   #'sample.txt' 
#'sample.txt'
path_archive_load = path_to_text_file_directory +'/'+ archive_file_name
#path_archive_load = archive_file_name
path_dict_load = 'D:\\archive'
dict_file_name = 'mydict.dict'
corpus_file_name = 'bow_corpus.mm'
#------------------------------
#Класс ReadTxtFiles принимает путь к каталогу содержащему текстовые файлы
class ReadTxtFiles(object):
    #Функция лемманизации

    def __init__(self, dirname):
        self.dirname = dirname
        
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname), encoding='utf-8'):   
                #Обращаю внимание на перекодировку - зависит от операционной системы. Желательно вынести в отдельную переменную
                #yield simple_preprocess(preprocess_text(line))  
                #Лемманизация сильно тормозит процесс!!! Возможно, что стоит её выделять в отдельный процесс с чтением-лемманизацией-записью
                
                yield simple_preprocess(line)    #Вариант без лемманизации
                # считывает файл построчно и использует функци gensim simple_preprocess для обработки одной строки файла за раз.
                # Переменная path_to_text_directory определяет путь до директории с файлами (можно подавать как input или передавать аргументом в исполняемую строку. 
                # Преимущество в том, что вы можете прочитать весь текстовый файл, не загружая файл в память сразу.)
#создание словаря (мешок слов)
mydict = corpora.Dictionary(ReadTxtFiles(path_to_text_file_directory))
# Для добавления новых файлов (в случае добавления новых файлов в жиректорию архива необходимо выполнить  dictionary.add_documents(навзание файла или списка файлов). Словарь будет обновлен
#токенизация и присваивание идентификаторов словам
#mydict.token2id
#print(mydict)


#Создание класса для корпуса слов (второй вариант - надо смотреть производительность)

#Класс Создание корпуса слов

class BoWCorpus(object):
    def __init__(self, path, dictionary):
        self.filepath = path
        self.dictionary = dictionary
    def __iter__(self):

        # читает строку из файла, обрабатывает ее с помощью simple_preprocess() и передает его в dictionary.doc2bow()
        
        global mydict  # Опционально, только если есть процесс добавления новых документов и необходимо обновлять словарь
        for line in smart_open(self.filepath, encoding='utf-8'):
            # Используется функцию smart_open() из пакета smart_open, так как оно позволяет построчно открывать 
            # и читать большие файлы из различных источников, таких 
            # как S3, HDFS, WebHDFS, HTTP или локальных и сжатых файлов.
            # Вызов функции токенизации
            #tokenized_list = simple_preprocess(preprocess_text(line), deacc=True)  #С лемманизацией
            tokenized_list = simple_preprocess(line, deacc=True)   #без лемманизации
            # Создание "мешка слов"
            bow = self.dictionary.doc2bow(tokenized_list, allow_update=True)
            # update the source dictionary (OPTIONAL)
            mydict.merge_with(self.dictionary)
            # 
            yield bow

# Создание корпуса
bow_corpus = BoWCorpus(path_archive_load, dictionary=mydict)  
# память свободна   Пока сделан запуск по одному файлу, на моем ноутбуке не тянет :)  Очень долго тогда
# Вывод token_id и количества в строку
#for line in bow_corpus:
#    print(line)
    
# Сохранение словаря и корпуса на диск
mydict.save(dict_file_name)  # сохранение словаря на диск
corpora.MmCorpus.serialize(corpus_file_name, bow_corpus)  # сохранение корпуса на диск

# Загрузка корпуса.  В принципе, здесь можно разделить полностью процессы в случае большого объема данных и распараллалить на 
# запись/чтение с диска для уменьшения нагрузки на память
loaded_dict = corpora.Dictionary.load(dict_file_name)

corpus = corpora.MmCorpus(corpus_file_name)
#for line in corpus:
#    print(line)

# Представление весов слов в корпусе (потом закоммитеть - не нужно)
#for doc in corpus:
#    print([[mydict[id], freq] for id, freq in doc])

# Создание TF-IDF модели
tfidf = models.TfidfModel(corpus, smartirs='ntc')

# Представление  TF-IDF весов (потом закоммитеть - не нужно)
for doc in tfidf[corpus]:
    print([[mydict[id], np.around(freq, decimals=2)] for id, freq in doc])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MSI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\MSI\AppData\Roaming\Python\Python37\site-packages\smart_open\smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


[['министерство', 0.57], ['природных', 0.47], ['ресурсов', 0.36], ['экологии', 0.57]]
[['федерации', 0.71], ['россиискои', 0.71]]
[['агентство', 0.69], ['по', 0.2], ['федеральное', 0.69]]
[['по', 0.26], ['департамент', 0.46], ['заказчик', 0.46], ['округу', 0.41], ['сибирскому', 0.41], ['федеральному', 0.41]]
[['акционерное', 0.48], ['ао', 0.27], ['исполнитель', 0.37], ['общество', 0.48], ['росгео', 0.39], ['росгеология', 0.42]]
[['отчет', 1.0]]
[['по', 0.23], ['объекту', 0.61], ['работ', 0.33], ['результатах', 0.68]]
[['выветривания', 0.23], ['золотоносные', 0.35], ['кемеровская', 0.43], ['коры', 0.26], ['на', 0.19], ['область', 0.43], ['площади', 0.21], ['поисковые', 0.34], ['пределах', 0.27], ['работы', 0.27], ['ольгинскои', 0.21]]
[['контракт', 0.66], ['от', 0.37], ['государственныи', 0.66]]
[['от', 0.28], ['бп', 0.55], ['кем', 0.55], ['лицензия', 0.55]]
[['книгах', 0.58], ['папках', 0.58], ['тетради', 0.58]]
[['книга', 0.57], ['отчета', 0.55], ['текст', 0.61]]
[['москва', 1.0]]
[['

[['ао', 0.11], ['работ', 0.08], ['на', 0.07], ['работы', 0.1], ['от', 0.1], ['буровые', 0.18], ['геодезические', 0.18], ['топографо', 0.18], ['фгуп', 0.12], ['цнигри', 0.12], ['осп', 0.12], ['пго', 0.12], ['сибирское', 0.12], ['полевых', 0.13], ['частично', 0.16], ['выполнялись', 0.16], ['силами', 0.15], ['виды', 0.2], ['грузов', 0.2], ['договор', 0.16], ['дорог', 0.2], ['основные', 0.2], ['полевыми', 0.2], ['работами', 0.15], ['связанных', 0.2], ['сентября', 0.16], ['строительство', 0.4], ['субподряда', 0.2], ['технологически', 0.2], ['транспортировка', 0.2], ['условиях', 0.16], ['здании', 0.2], ['сооружении', 0.2]]
[['работ', 0.25], ['полевых', 0.42], ['принимали', 0.51], ['проведении', 0.49], ['участие', 0.51]]
[['исполнитель', 0.14], ['геологическая', 0.07], ['арнаутова', 0.14], ['бикмулин', 0.17], ['бородина', 0.17], ['ведущие', 0.14], ['виснап', 0.17], ['власов', 0.17], ['волков', 0.15], ['геолог', 0.14], ['геологи', 0.43], ['дмитриев', 0.17], ['жердецкая', 0.14], ['кат', 0.41], 

In [26]:
# Human readable format of corpus (term-frequency)
[[(loaded_dict[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('министерство', 1.0),
  ('природных', 1.0),
  ('ресурсов', 1.0),
  ('экологии', 1.0)]]